## Doctor Right


#### Import Library

In [1]:
import sys
sys.path.append("../modules")
from eda import EDAAnalyzer
from spark_session import SparkManager
from feature_engineering import FeatureEngineer

In [2]:
# Load autoreload extension
%load_ext autoreload
%autoreload 2

#### Constants and config

In [3]:
mx_submits_path = "../data_sample/mx_submits.parquet/"
mx_submits_line_path = "../data_sample/mx_submitsline.parquet/"

### Spark Session

In [4]:
mx_submits_spark_manager = SparkManager(mx_submits_path)
mx_submits_line_spark_manager = SparkManager(mx_submits_line_path)

24/09/15 17:41:21 WARN Utils: Your hostname, Sureshs-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 172.20.9.214 instead (on interface en0)
24/09/15 17:41:21 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/09/15 17:41:21 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


### MX SUBMITS

#### EDA

In [5]:
mx_submits_eda = EDAAnalyzer(mx_submits_spark_manager)

In [6]:
mx_submits_eda.display_head()

24/09/15 17:41:25 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


,claim_number,cohort_key,cohort_record_id,attending_provider_npi,billing_provider_address_cbsa_name,billing_provider_address_city,billing_provider_address_county,billing_provider_address_key,billing_provider_address_latitude,billing_provider_address_longitude,...,type_of_bill_frequency_description,vendorname,cycle_id,patient_birth_year,patient_gender,patient_id,patient_location_preferred_latitude,patient_location_preferred_longitude,patient_location_preferred_zipcode,patient_location_preferred_type
0,d3f3ebb5cee7ccf1b9c3876060668c99,0e6b9e4a0d9103c5a9c4f1332d38ef53,afc38aa34fb64bebe04f15507e07ba17,1376869032,None,None,None,None,None,None,...,Interim (Continuing Claims),cyan,full_20240611,1964,F,8aad41f612a7095449888c8050abaeb05fdee65643caa3...,38.643135,-87.360331,47557,2.0
1,fced2c3b9dda8739b50b954b662d0e5e,0e6b9e4a0d9103c5a9c4f1332d38ef53,afc38aa34fb64bebe04f15507e07ba17,1376869032,None,None,None,None,None,None,...,Admit Through Discharge Claim,cyan,full_20240611,1964,F,8aad41f612a7095449888c8050abaeb05fdee65643caa3...,38.643135,-87.360331,47557,2.0
2,eeaaf85581b99f45028afa82b3c216f,0e6b9e4a0d9103c5a9c4f1332d38ef53,27c4b38fbb414c1d29231d9301214e52,None,"NORTH PORT-SARASOTA-BRADENTON, FL",Sarasota,Sarasota,943SBENEVARDSARASOTAFL34232,27.325453,-82.496513,...,Admit Through Discharge Claim,cyan,full_20240611,1962,U,5f1de4e894f8384574e789503276dbf4a759b40f225744...,27.329646,-82.502262,34237,1.0
3,c98ecf0bd97acc704af129d551ec9b30,0e6b9e4a0d9103c5a9c4f1332d38ef53,439846c6f66dd5939caa42094e613c7b,1598100059,None,None,None,None,None,None,...,Admit Through Discharge Claim,cyan,full_20240611,1976,F,1f9d89fd6a5cd3fdfcb387fef2c005300ca79ae5fa32ef...,39.946129,-83.815872,45504,1.0
4,c8223c08c26ee4d491bd3b17be802641,0e6b9e4a0d9103c5a9c4f1332d38ef53,db2cf4c86c3984123bfbfaef70af9f66,1659514149,None,None,None,None,None,None,...,Admit Through Discharge Claim,cyan,full_20240611,1960,F,69643c3b36d8f6df6ca79a72251f728f5fd366007a62d3...,None,None,None,NaN


In [7]:
mx_submits_eda.display_shape()

'Shape of data: rows: 247334, cols: 127'

In [ ]:
column_info_submits = mx_submits_eda.display_column_info()
column_info_submits.to_csv("../output/column_info_submits.csv")
column_info_submits

### MX SUBMITS LINE

#### EDA

In [ ]:
mx_submits_line_eda = EDAAnalyzer(mx_submits_line_spark_manager)

In [ ]:
mx_submits_line_eda.display_head()

In [ ]:
mx_submits_line_eda.display_shape()

In [ ]:
column_info_submits_line = mx_submits_line_eda.display_column_info()
column_info_submits_line.to_csv("../output/column_info_submits_line.csv")
column_info_submits_line

### Feature Engineering

In [ ]:
mx_submits_fe = FeatureEngineer(mx_submits_spark_manager)

In [ ]:
mx_submits_fe.add_comorbidities_array()

In [ ]:
%autoreload 2